In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import os
import pickle


TensorFlow 1.x selected.


In [ ]:
!mkdir images

In [ ]:
path="/content/drive/My Drive/"
path_data=os.path.join(path, "Task_Rest_Th0.7.rar")

In [ ]:
!unrar x "/content/drive/My Drive/Task_Rest_Th0.7.rar" "/content/images/"


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/drive/My Drive/Task_Rest_Th0.7.rar

Creating    /content/images/Train_Rest_Th0.7                          OK
Creating    /content/images/Train_Rest_Th0.7/test_Rest_Th0.7          OK
Extracting  /content/images/Train_Rest_Th0.7/test_Rest_Th0.7/Rest_0_6371.png       0%  OK 
Extracting  /content/images/Train_Rest_Th0.7/test_Rest_Th0.7/Rest_0_6396.png       0%  OK 
Extracting  /content/images/Train_Rest_Th0.7/test_Rest_Th0.7/Rest_0_6422.png       0%  OK 
Extracting  /content/images/Train_Rest_Th0.7/test_Rest_Th0.7/Rest_0_6522.png       0%  OK 
Extracting  /content/images/Train_Rest_Th0.7/test_Rest_Th0.7/Rest_0_6662.png       0%  OK 
Extracting  /content/images/Train_Rest_Th0.7/test_Rest_Th0.7/Rest_0_6712.png       0%  OK 
Extracting  /content/images/Train_Rest_Th0.7/test_Rest_Th0.7/Rest_0_6757.png       0%  OK 
Extracting  /content/ima

In [ ]:
path_img="/content/images/Train_Rest_Th0.7"
dirs = os.listdir(path_img)

df_train_task=pd.DataFrame({"img":[cv2.resize(cv2.imread(os.path.join(path_img, dirs[0], i)), (228, 228)) for i in os.listdir(os.path.join(path_img, dirs[0]))], "label":[1]*len(os.listdir(os.path.join(path_img, dirs[0]))), "istrain":[1]*len(os.listdir(os.path.join(path_img, dirs[0]))) })
df_train_rest=pd.DataFrame({"img":[cv2.resize(cv2.imread(os.path.join(path_img, dirs[1], i)), (228, 228)) for i in os.listdir(os.path.join(path_img, dirs[1]))], "label":[0]*len(os.listdir(os.path.join(path_img, dirs[1]))), "istrain":[1]*len(os.listdir(os.path.join(path_img, dirs[1]))) })
df_test_rest=pd.DataFrame({"img":[cv2.resize(cv2.imread(os.path.join(path_img, dirs[2], i)), (228, 228)) for i in os.listdir(os.path.join(path_img, dirs[2]))], "label":[0]*len(os.listdir(os.path.join(path_img, dirs[2]))), "istrain":[0]*len(os.listdir(os.path.join(path_img, dirs[2]))) })
df_test_task=pd.DataFrame({"img":[cv2.resize(cv2.imread(os.path.join(path_img, dirs[3], i)), (228, 228)) for i in os.listdir(os.path.join(path_img, dirs[3]))], "label":[1]*len(os.listdir(os.path.join(path_img, dirs[3]))), "istrain":[0]*len(os.listdir(os.path.join(path_img, dirs[3]))) })

df_train=pd.concat([df_train_rest, df_train_task]).reset_index(drop=True)
df_test=pd.concat([df_test_rest, df_test_task]).reset_index(drop=True)

df = pd.concat([df_train, df_test]).reset_index(drop=True)

In [ ]:
os.listdir(path)

['car_landmarks',
 'Copy of car_landmarks',
 'ICPR',
 'raw_data',
 'Task_Rest_Th0.7.rar',
 'best_split_Th06Sep17.pkl',
 'BCT_IEEETranS1.h5',
 'BCT_IEEETranS1.hdf5',
 'wind_shield',
 'ICPRROC_AllUsers.pdf',
 'Colab Notebooks',
 'cifar-10-python.tar.gz',
 'MWL_BrainConnectivity_GraphCNN-LSTM.ipynb',
 'best_split_Shubhasis_Th07.pkl',
 'IMP_MultipleTransferModel_LSTM.pdf',
 'IMP_TransPaper_Tempo_Spatio_SpectralDeepModel.pdf']

In [ ]:
import pickle
with open("/content/drive/My Drive/best_split_Shubhasis_Th07.pkl", "rb") as f:
  best_split=pickle.load(f)

In [ ]:
train_x, train_y, test_x, test_y=[], [], [], []
seq=2
train_data=df.iloc[best_split["train"]] ##row access from best split (upper cell)
test_data=df.iloc[best_split["test"]]
for i in range(train_data.shape[0]-seq):
  train_x.append(train_data.iloc[i:i+seq].img)
  train_y.append(train_data.iloc[i:i+seq].label)
for i in range(test_data.shape[0]-seq):
  test_x.append(test_data.iloc[i:i+seq].img)
  test_y.append(test_data.iloc[i:i+seq].label)
train_x, train_y, test_x, test_y = np.array(train_x), np.array(train_y).reshape(len(train_y), seq), np.array(test_x), np.array(test_y).reshape(len(test_y), seq)

In [ ]:
##Model 2: Time distributed CNN-LSTM
import keras
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Dropout, Input, Lambda,Flatten
from keras.layers import Conv1D, MaxPooling1D, Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.regularizers import l2
from keras.optimizers import RMSprop, Adadelta
from keras import optimizers
from keras import backend as K
import matplotlib.pyplot as plt
from keras import regularizers
import cv2
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import Model
#from matplotlib import pyplot
from numpy import expand_dims
from keras.callbacks import LearningRateScheduler
from keras.layers import TimeDistributed
from keras.layers import Dense, Dropout, LSTM,BatchNormalization
from keras.utils.vis_utils import plot_model

timesteps,row,col,chnls = train_x.shape[-4], train_x.shape[-3], train_x.shape[-2], train_x.shape[-1]


cnn=keras.models.Sequential()
cnn.add(keras.layers.TimeDistributed(keras.layers.Conv2D(8, kernel_size=(9,9), activation='relu', 
                                                         #kernel_regularizer=regularizers.l2(1e-1),
                                                        #  bias_regularizer=regularizers.l2(1e-1),
                                                        #  activity_regularizer=regularizers.l2(1e-1),
                                                         data_format="channels_last"),input_shape=(timesteps,row,col,chnls)))
cnn.add(BatchNormalization(epsilon=1e-04, momentum=0.9))
cnn.add(keras.layers.TimeDistributed(keras.layers.MaxPooling2D(pool_size=3)))

cnn.add(keras.layers.TimeDistributed(keras.layers.Conv2D(16, kernel_size =(7,7),activation='relu'
                                                         #kernel_regularizer=regularizers.l2(1e-1)
                                                        #  bias_regularizer=regularizers.l2(1e-1),
                                                        #  activity_regularizer=regularizers.l2(1e-1)
                                                        )))
cnn.add(BatchNormalization(epsilon=1e-04, momentum=0.9))
cnn.add(keras.layers.TimeDistributed(keras.layers.MaxPooling2D(pool_size=3)))

cnn.add(keras.layers.TimeDistributed(keras.layers.Conv2D(32, kernel_size =(5,5),activation='relu'
                                                         #kernel_regularizer=regularizers.l2(1e-1)
                                                        #  bias_regularizer=regularizers.l2(1e-1),
                                                        #  activity_regularizer=regularizers.l2(1e-1)
                                                         )))
cnn.add(BatchNormalization(epsilon=1e-04, momentum=0.9))
cnn.add(keras.layers.TimeDistributed(keras.layers.MaxPooling2D(pool_size=3)))

# cnn.add(keras.layers.TimeDistributed(keras.layers.Conv2D(32, kernel_size =(3,3),activation='relu')))
# cnn.add(BatchNormalization(epsilon=1e-04, momentum=0.9))
# cnn.add(keras.layers.TimeDistributed(keras.layers.MaxPooling2D(pool_size=3)))

cnn.add(keras.layers.TimeDistributed(keras.layers.Flatten()))

cnn.add(LSTM(units=40,return_sequences=True,activation='relu'))
cnn.add(BatchNormalization(epsilon=1e-04, momentum=0.9))
cnn.add(Dropout(0.2))

cnn.add(LSTM(units=20, return_sequences=False,activation='relu'))
cnn.add(BatchNormalization(epsilon=1e-04, momentum=0.9))
cnn.add(Dropout(0.2))

cnn.add(Dense(10, activation='relu'))
cnn.add(BatchNormalization(epsilon=1e-04, momentum=0.9))
cnn.add(Dense(1, activation='sigmoid'))
adam=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=0.00001, decay=0.001)
#cnn.compile(loss=binary_focal_loss_fixed,optimizer=adam, metrics=['categorical_accuracy'])
cnn.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
#cnn.compile(loss='categorical_crossentropy',optimizer=adam, metrics=['categorical_accuracy'])
cnn.summary()
# plot_model(cnn, to_file=path+'BCT_2DCNN-LSTM_model_plot.png', show_shapes=True, show_layer_names=True)



Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Using TensorFlow backend.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 2, 220, 220, 8)    1952      
_________________________________________________________________
batch_normalization_1 (Batch (None, 2, 220, 220, 8)    32        
_________________________________________________________________
time_distributed_2 (TimeDist (None, 2, 73, 73, 8)      0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 2, 67, 67, 16)     6288      
_________________________________________________________________
batch_normalization_2 (Batch (None, 2, 67, 67, 16)     64        
_________________________________________________________________
time_distributed_4 (TimeDist (None, 2, 22, 22, 16)     0         
_________________________

In [ ]:
model = tf.keras.models.load_model(os.path.join(path, "BCT_IEEETranS1_Model.h5"))

In [ ]:
for layer in model.layers:
  layer.trainable=False
  print(layer.name)

time_distributed_19
time_distributed_20
time_distributed_21
time_distributed_22
time_distributed_23
lstm_6
batch_normalization_8
lstm_7
batch_normalization_9
dense_5
batch_normalization_10
dense_6


In [ ]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_19 (TimeDis (None, 2, 228, 228, 32)   896       
_________________________________________________________________
time_distributed_20 (TimeDis (None, 2, 114, 114, 32)   0         
_________________________________________________________________
time_distributed_21 (TimeDis (None, 2, 114, 114, 48)   13872     
_________________________________________________________________
time_distributed_22 (TimeDis (None, 2, 57, 57, 48)     0         
_________________________________________________________________
time_distributed_23 (TimeDis (None, 2, 155952)         0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 2, 30)             18717960  
_________________________________________________________________
batch_normalization_8 (Batch (None, 2, 30)           

In [ ]:
def SpatialFeatures():
  return tf.keras.Model(model.input,  [model.layers[0].output, model.layers[2].output])

In [ ]:
model.predict([:2])

array([[0.],
       [0.]], dtype=float32)

In [ ]:
model.save_weights("weights.h5")

In [ ]:
FtrsModel = SpatialFeatures()
FtrsModel.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])
FtrsModel.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_19_input (I [(None, 2, 228, 228, 3)]  0         
_________________________________________________________________
time_distributed_19 (TimeDis (None, 2, 228, 228, 32)   896       
_________________________________________________________________
time_distributed_20 (TimeDis (None, 2, 114, 114, 32)   0         
_________________________________________________________________
time_distributed_21 (TimeDis (None, 2, 114, 114, 48)   13872     
Total params: 14,768
Trainable params: 0
Non-trainable params: 14,768
_________________________________________________________________


In [ ]:
features = FtrsModel.predict(train_x[:1])

In [ ]:
features[0].shape

(1, 2, 228, 228, 32)